<a href="https://colab.research.google.com/github/SilvanusTurkson/Face-Reg/blob/main/new_face_recognition_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install deepface opencv-python pandas
!pip install streamlit deepface opencv-python scikit-learn numpy pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [8]:
import cv2
from deepface import DeepFace
import pandas as pd
import time

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Set the resolution (optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Create a database of known faces (replace with your own images)
# Format: {"name": "path_to_image.jpg"}
known_faces = {
    "Person1": "/content/drive/MyDrive/images/5.jpg",
    "Person2": "/content/drive/MyDrive/images/6.jpg"
}

# Verify the known faces and create embeddings
face_database = []
for name, img_path in known_faces.items():
    try:
        embedding = DeepFace.represent(img_path=img_path, model_name="Facenet")[0]["embedding"]
        face_database.append({"name": name, "embedding": embedding})
        print(f"Successfully added {name} to the database")
    except Exception as e:
        print(f"Error processing {name}: {str(e)}")

# Threshold for face recognition (adjust as needed)
THRESHOLD = 0.4

# For FPS calculation
prev_time = 0
curr_time = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Detect faces in the frame
        detections = DeepFace.extract_faces(frame, detector_backend="opencv", enforce_detection=False)

        for detection in detections:
            if detection["confidence"] > 0.9:  # Only consider confident detections
                x, y, w, h = detection["facial_area"]["x"], detection["facial_area"]["y"], \
                              detection["facial_area"]["w"], detection["facial_area"]["h"]

                # Draw rectangle around the face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Extract the face region
                face_img = frame[y:y+h, x:x+w]

                try:
                    # Get embedding for the detected face
                    detected_embedding = DeepFace.represent(img_path=face_img, model_name="Facenet", enforce_detection=False)
                    if detected_embedding:
                        detected_embedding = detected_embedding[0]["embedding"]

                        # Compare with known faces
                        min_dist = float("inf")
                        identity = "Unknown"

                        for person in face_database:
                            distance = DeepFace.distance(detected_embedding, person["embedding"], model_name="Facenet")
                            if distance < min_dist and distance < THRESHOLD:
                                min_dist = distance
                                identity = person["name"]

                        # Display the identity
                        cv2.putText(frame, identity, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                except Exception as e:
                    print(f"Error in recognition: {str(e)}")

    except Exception as e:
        print(f"Error in face detection: {str(e)}")

    # Calculate and display FPS
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    prev_time = curr_time
    cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow("Real-Time Face Recognition", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Successfully added Person1 to the database
Successfully added Person2 to the database


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
